In [1]:
import requests
import json
import datetime as dt
from hdfs import InsecureClient
from bs4 import BeautifulSoup
client = InsecureClient('http://localhost:9870', user='big') #나의 로컬호스트주소 #user : big(내가정함)

In [2]:
# 라이브러리 입장에서는 404도 응답이기 때문에 에러가 아니다. 
# 우리가 그런상황도 error라고 정해 놓고 가된다.

# raise Exception('응답코드 : ' + str(404))  의도적 에러 Exception: 응답코드 : 404

def execute_rest_api(method, url, headers, params):
    if method == 'get':
        res = requests.get(url,params=params,headers=headers)
    elif method == 'post' :
        res = requests.post(url,params=params,headers=headers)
    
    if res ==None or res.status_code != 200 :
        raise Exception('응답코드 : ' + str(res.status_code))
        
    return res.text

In [3]:
def cal_std_day(befor_day):   
    x = dt.datetime.now() - dt.timedelta(befor_day)
    year = x.year
    month = x.month if x.month >= 10 else '0'+ str(x.month)
    day = x.day if x.day >= 10 else '0'+ str(x.day)  
    return str(year)+ '-' +str(month)+ '-' +str(day)

In [4]:
# 로그만들기
import logging
# 내가원하는 과정중에 발생한 에러만 골라서 기록하는 로깅이 필요
co_logger = logging.getLogger('corona_api')
# 로그를 한파일에만 넣으면 나중에 찾기 힘들고 심지어 열기조차 불가능 할 수 있음 -> 이번엔 날짜별로 하자, 하둡은 파일 크기별로함
handler = logging.FileHandler('./log/rest_api/'+cal_std_day(0)+'.log')
co_logger.addHandler(handler)

In [5]:
# 시장지표 크롤링
base_url = 'https://finance.naver.com/marketindex/'
# 환전고시, 유가/금시세, 원자재
params = ['?tabSel=exchange#tab_section', '?tabSel=gold#tab_section', '?tabSel=materials#tab_section']

url = base_url + params[0]

In [6]:
from datetime import date, datetime
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pandas as pd

<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)


In [7]:
## oil_gold
def get_oil_gold(before_day=0):
    # 시장지표 크롤링
    base_url = 'https://finance.naver.com/marketindex/'
    # 환전고시, 유가/금시세, 원자재
    params = ['?tabSel=exchange#tab_section', '?tabSel=gold#tab_section', '?tabSel=materials#tab_section']
    file_dir = '/finance/oil_gold/'
    file_name =[ 'oil_price_' + str(cal_std_day(before_day))+'.json', 'gold_price_' + str(cal_std_day(before_day))+'.json' ]
    url = base_url + params[1]
    response_txt = execute_rest_api('get',url,{},{})
    soup = BeautifulSoup(response_txt, 'html.parser')
    ops_list = [soup.find('div',{'id':'content'}).findAll('div',{'class':'section_exchange'})[0].findAll('tr')[1:],
         soup.find('div',{'id':'content'}).findAll('div',{'class':'section_exchange'})[1].findAll('tr')[1:] ]

    cols = ['product_name','unit','price','fluctuation_rate','is_rise']


    data_oil = []
    data_gold = []
    rows = []
    for idx,ops in enumerate(ops_list) :
        for op in ops:
            # 유가
            if idx ==  0:
                rows = []
                # 상품명, 단위명, 현재가, 등락율, 상승여부 : True는 상승
                rows.append(op.findAll('td')[0].text)
                rows.append(op.findAll('td')[1].text)
                rows.append(op.findAll('td')[2].text.replace(',',''))
                rows.append(op.findAll('td')[-2].text[:-1].replace(' ','').replace('%','').replace('+','').replace('-',''))
                if op['class'][0] == 'down' :
                    rows.append(False)
                    tmp = dict(zip(cols,rows))
                    data_oil.append(tmp)
                else :
                    rows.append(True)
                    tmp = dict(zip(cols,rows))
                    data_oil.append(tmp)
            # 귀금속
            if idx == 1 :
                rows=[]
                # 상품명, 단위명, 현재가, 등락율, 상승여부 : True는 상승
                rows.append(op.findAll('td')[0].text)
                rows.append(op.findAll('td')[1].text)
                rows.append(op.findAll('td')[2].text.replace(',',''))
                rows.append(op.findAll('td')[-2].text.replace(' ','').replace('%','').replace('+','').replace('-',''))
                if op['class'][0] == 'down' :
                    rows.append(False)
                    tmp = dict(zip(cols,rows))
                    data_gold.append(tmp)
                else :
                    rows.append(True)
                    tmp = dict(zip(cols,rows))
                    data_gold.append(tmp)

    res_oil = {
        'meta':{
            'desc':'국내 및 국제 유가',
            'cols':{
            'product_name':'상품명',
            'unit':'단위',
            'price':'현재가',
            'fluctuation_rate':'등락율',
            'is_rise':'전일대비상승여부'
            },
            'std_day':cal_std_day(before_day),
            'product_line':'oil'
        },
        'data':data_oil
    }
    res_gold = {
        'meta':{
            'desc':'귀금속 국내 및 국제가격',
            'cols':{
            'product_name':'상품명',
            'unit':'단위',
            'price':'현재가',
            'fluctuation_rate':'등락율',
            'is_rise':'전일대비상승여부'
            },
            'std_day':cal_std_day(before_day),
            'product_line':'precious_metal'
        },
        'data':data_gold
    }
    # 유가정보 저장
    client.write(file_dir+file_name[0],json.dumps(res_oil,ensure_ascii=False),encoding='utf-8')
    # 귀금속정보 저장
    client.write(file_dir+file_name[1],json.dumps(res_gold,ensure_ascii=False),encoding='utf-8')

In [8]:
# 매일매일 환율 가져오기


def get_exchange(before_day=0) :
    # 시장지표 크롤링
    base_url = 'https://finance.naver.com/marketindex/'
    # 환전고시, 유가/금시세, 원자재
    params = ['?tabSel=exchange#tab_section', '?tabSel=gold#tab_section', '?tabSel=materials#tab_section']
    file_dir = '/finance/exchange/'
    file_name = 'sale_standard_rate_' + str(cal_std_day(before_day))+'.json'
    url = base_url + params[0]
    response_txt = execute_rest_api('get',url,{},{})
    soup = BeautifulSoup(response_txt, 'html.parser')
    ops = soup.find('select',{'id':'select_to'}).findAll('option')[2:]
    cols = ['country_name','exr_mont_unit','exr_stad_rate']

    data = []
    for op in ops:
        rows = []
        # 국가, 화폐이름(영어) 만필요
        # 국가이름 : country_name
        rows.append(op.text.split(' ')[0])
        # 화폐이름 : exr_mont_unit
        rows.append(op.text.split(' ')[-1])
        #매매기준율 : exr_stad_rate
        rows.append(op['value'])
        tmp = dict(zip(cols,rows))
        data.append(tmp)

    res = {
        'meta':{
            'desc':'국가별 매매기준율 1국가화폐당 원화',
            'cols':{
            'country_name':'국가명',
            'exr_mont_unit':'화폐이름',
            'exr_stad_rate':'매매기준율',
            },
            'std_day':cal_std_day(before_day)
        },
        'data':data
    }
    
    client.write(file_dir+file_name,json.dumps(res,ensure_ascii=False),encoding='utf-8')

In [9]:
# 원자재
# 시장지표 크롤링
base_url = 'https://finance.naver.com/marketindex/'
# 환전고시, 유가/금시세, 원자재
params = ['?tabSel=exchange#tab_section', '?tabSel=gold#tab_section', '?tabSel=materials#tab_section']
file_dir = '/finance/raw_materials/'
file_name =[ 'energy_' + str(cal_std_day(0))+'.json', 'non_metal_' + str(cal_std_day(0))+'.json',
           'agriculture_' + str(cal_std_day(0))+'.json' ]
url = base_url + params[2]
response_txt = execute_rest_api('get',url,{},{})
soup = BeautifulSoup(response_txt, 'html.parser')
soup_div = soup.find('div',{'id':'content'}).findAll('div',{'class':'section_material'})
ops_list = [soup_div[0].findAll('tr')[1:],soup_div[1].findAll('tr')[1:],soup_div[2].findAll('tr')[1:]]

cols=['product_name','current_month_contract','unit','price','fluctuation_rate','futures_exchange_name','is_rise']
cols_non_metal =['product_name','unit','price','fluctuation_rate','futures_exchange_name','is_rise']
data_energy = []
data_non_metal = []
data_agriculture = []



for idx, ops in enumerate(ops_list):
    for op in ops:
        if idx == 0 :
            # 에너지 선물
            # 상품명, 월물, 단위, 현재가, 등락율, 거래소명, 상승여부(전일대비) : True -> 상승
            rows = []
            rows.append(op.findAll('td')[0].text)
            rows.append(op.findAll('td')[1].text.strip())
            rows.append(op.findAll('td')[2].text)
            rows.append(op.findAll('td')[3].text.replace(',',''))
            rows.append(op.findAll('td')[5].text.replace('%','').replace('+','').replace('-',''))
            rows.append(op.findAll('td')[7].text)
            if op['class'][0] == 'down' :
                rows.append(False)
                tmp = dict(zip(cols,rows))
                data_energy.append(tmp)
            else :
                rows.append(True)
                tmp = dict(zip(cols,rows))
                data_energy.append(tmp)
        if idx == 1 :
            rows=[]
            # 비철금속 현물
            #상품명, 단위, 현재가, 등락율, 거래소명, 상승여부(전일대비) : True -> 상승
            rows.append(op.findAll('td')[0].text)
            rows.append(op.findAll('td')[1].text)
            rows.append(op.findAll('td')[2].text.replace(',',''))
            rows.append(op.findAll('td')[4].text.replace('%','').replace('+','').replace('-',''))
            rows.append(op.findAll('td')[6].text)
            if op['class'][0] == 'down' :
                rows.append(False)
                tmp = dict(zip(cols_non_metal,rows))
                data_non_metal.append(tmp)
            else :
                rows.append(True)
                tmp = dict(zip(cols_non_metal,rows))
                data_non_metal.append(tmp)
                
        if idx ==2 :
        # 농산물 선물
        # 상품명, 월물, 단위, 현재가, 등락율, 거래소명, 상승여부(전일대비) : True -> 상승
            rows = []
            rows.append(op.findAll('td')[0].text)
            rows.append(op.findAll('td')[1].text.strip())
            rows.append(op.findAll('td')[2].text)
            rows.append(op.findAll('td')[3].text.replace(',',''))
            rows.append(op.findAll('td')[5].text.replace('%','').replace('+','').replace('-',''))
            rows.append(op.findAll('td')[7].text)
            if op['class'][0] == 'down' :
                rows.append(False)
                tmp = dict(zip(cols,rows))
                data_agriculture.append(tmp)
            else :
                rows.append(True)
                tmp = dict(zip(cols,rows))
                data_agriculture.append(tmp)

res_energy = {
    'meta':{
        'desc':'에너지 선물',
        'cols':{
            'product_name':'상품명','current_month_contract':'월물','unit':'단위','price':'현재가',
            'fluctuation_rate':'등락율','futures_exchange_name':'선물거래소명','is_rise':'상승여부'
        },
        'std_day':cal_std_day(0),
        'product_line':'energy'
    },
    'data':data_energy
}
res_non_metal = {
    'meta':{
        'desc':'비철금속 현물',
        'cols':{
            'product_name':'상품명','unit':'단위','price':'현재가',
            'fluctuation_rate':'등락율','futures_exchange_name':'선물거래소명','is_rise':'상승여부'
        },
        'std_day':cal_std_day(0),
        'product_line':'non_metal'
    },
    'data':data_non_metal
}
res_agriculture = {
    'meta':{
        'desc':'농산물 선물',
        'cols':{
            'product_name':'상품명','current_month_contract':'월물','unit':'단위','price':'현재가',
            'fluctuation_rate':'등락율','futures_exchange_name':'선물거래소명','is_rise':'상승여부'
        },
        'std_day':cal_std_day(0),
        'product_line':'agriculture'
    },
    'data':data_agriculture
}
res_list = [res_energy,res_non_metal,res_agriculture]
for idx,file_na in enumerate(file_name):
    client.write(file_dir+file_na,json.dumps(res_list[idx],ensure_ascii=False),encoding='utf-8')

In [10]:
get_oil_gold(0)
get_exchange(0)